In [1]:
# from utility.RetroParser import RetroData
from utility.RetroGateModel import RetroRLModel
from utility.RetroRLAgent import RetroRLAgent
# from utility.ResultProcess import ResultParser
from utility.DataPrepare import Prepare
from utility.BruteForceSearch import expansion, Product, Reaction
import time
import numpy as np
# # Use Braket SDK Cost Tracking to estimate the cost to run this example
# from braket.tracking import Tracker
# t = Tracker().start()

timestamp = time.strftime("%Y%m%d-%H")

INFO:rdkit:Enabling RDKit 2022.03.5 jupyter extensions


In [2]:
# config your aws account in your ~/.aws/config
import os
os.environ['AWS_DEFAULT_REGION']='us-west-1'

# Step 1: Prepare Data

In this part, we load the retrosynthesis prediction data for experiment.
The [USPTO-50K](https://tdcommons.ai/generation_tasks/retrosyn/#uspto-50k) was 
put in the repository. We assign the relative 
path to **raw_path**.
The **s3_bucket** and **prefix** are used to store the 
results. We can use the one created with the 
cloudformation for convenience.

In [ ]:
# input: predata_uspto-50k.xlsx
# output: file1.npy,file2.npy
raw_path = './uspto50k.xlsx'
prepare = Prepare(raw_path)
prepare.generate_files()  # 
prepare.generate_ground_truth()
ground_truth = np.load(prepare.path+'ground_truth.npy', allow_pickle=True).tolist()

INFO:root:parse xlsx file!
INFO:root:There are at most 4 reactants in one reaction!
INFO:root:Start picking dataset.
  8%|█████▉                                                                     | 3983/50037 [09:18<1:51:25,  6.89it/s]

In [ ]:
data_path = 'data'
!mkdir $data_path
# !cp buyable.npy $data_path
# !cp deadend.npy $data_path
# !cp reactions_dictionary.npy $data_path
# !cp smiles_dictionary.npy $data_path
# !cp target_product.npy $data_path
# !cp ground_truth.npy $data_path

# windows
!copy buyable.npy $data_path
!copy deadend.npy $data_path
!copy reactions_dictionary.npy $data_path
!copy smiles_dictionary.npy $data_path
!copy target_product.npy $data_path
!copy ground_truth.npy $data_path

# Step 2: Build Model

In this part, we build the circuit model for retrosynthetic planning


In [ ]:
# initial the RetroRLModel object
init_param = {}
method = ['retro-rl', 'retro-qrl']

for mt in method:
    if mt == 'retro-rl':
        init_param[mt] = {}
        init_param[mt]['param'] = ['inputsize', 'middlesize', 'outputsize']
    elif mt == 'retro-qrl':
        init_param[mt] = {}
        init_param[mt]['param'] = ['n_qubits', 'device', 'framework', 'shots']
    
retro_rl_model = RetroRLModel(data=None, method=method, **init_param)

In [ ]:
model_param={}
method = 'retro-rl'
model_param[method] = {}
model_param[method]['inputsize'] = [256]
model_param[method]['middlesize'] = [512]
model_param[method]['outputsize'] = [1]

retro_rl_model.build_model(**model_param)

In [ ]:
model_param={}
method = 'retro-qrl'
model_param[method] = {}
model_param[method]['n_qubits'] = [8]
model_param[method]['device'] = ['local', 'sv1', 'aspen-m2']
model_param[method]['framework'] = ['pennylane']
model_param[method]['shots'] = [100]

retro_rl_model.build_model(**model_param)


We can use the following method to check the properties of 
model. This way, we can build many models conveniently. 
After that, we save the model and update the value of 
**model_path**.

In [ ]:
# describe the model parameters
model_info = retro_rl_model.describe_model()

In [ ]:
# save the model
model_path = retro_rl_model.save("latest")

print(f"You have built the nn model for RL and saved it as {model_path}")

In [ ]:
# !cp $model_path $data_path

# windows
!copy $model_path $data_path

# Step 3: Learn Retrosynthetic Planning

In this part, we use cpu to run classical model for retrosynthetic planning 
and simulators/NISQ devices to run quantum model for retrosysnthetic planning.

In [ ]:
model_path='./retrorl_model_latest.pickle'

# get the model you want to optimize
n_qubits = 8
device = 'sv1'
framework = 'pennylane'
shots = 100

model_name = "{}_{}_{}_{}".format(n_qubits, device, framework, shots)
method = "retro-qrl"

In [ ]:
# train_mode can be: "local-instance", "local-job", "hybrid-job"
train_mode = "hybrid-job"

In [ ]:
data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_model = None
if train_mode == "local-instance":
    # get model
    retro_rl_model = RetroRLModel.load(model_path)
    model_info = retro_rl_model.describe_model()
    retro_model = retro_rl_model.get_model(method, model_name)

retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()

job_arn = retro_rl_agent.get_job_arn()
print(f"create job with arn {job_arn}")

In [ ]:
retro_rl_agent.get_job().cancel()

In [ ]:
# get the model you want to optimize
n_qubits = 8
device = 'local'
framework = 'pennylane'
shots = 100

model_name = "{}_{}_{}_{}".format(n_qubits, device, framework, shots)
method = "retro-qrl"

data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_model = None
if train_mode == "local-instance":
    # get model
    retro_rl_model = RetroRLModel.load(model_path)
    model_info = retro_rl_model.describe_model()
    retro_model = retro_rl_model.get_model(method, model_name)

retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()

job_arn = retro_rl_agent.get_job_arn()
print(f"create job with arn {job_arn}")

In [ ]:
retro_rl_agent.get_job().cancel()

In [ ]:
# get the model you want to optimize
# config your aws account in your ~/.aws/config
import os
os.environ['AWS_DEFAULT_REGION']='us-west-1'

n_qubits = 8
device = 'aspen-m2'
framework = 'pennylane'
shots = 100

model_name = "{}_{}_{}_{}".format(n_qubits, device, framework, shots)
method = "retro-qrl"

data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_model = None
if train_mode == "local-instance":
    # get model
    retro_rl_model = RetroRLModel.load(model_path)
    model_info = retro_rl_model.describe_model()
    retro_model = retro_rl_model.get_model(method, model_name)

retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()

job_arn = retro_rl_agent.get_job_arn()
print(f"create job with arn {job_arn}")

In [ ]:
retro_rl_agent.get_job().cancel()

In [ ]:
model_path='./retrorl_model_latest.pickle'

# get the model you want to optimize
inputsize = 256
middlesize = 512
outputsize = 1

model_name = "{}_{}_{}".format(inputsize, middlesize, outputsize)
method = "retro-rl"

# train_mode can be: "local-instance", "local-job", "hybrid-job"
train_mode = "hybrid-job"

In [ ]:
data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_rl_model = RetroRLModel.load(model_path)
model_info = retro_rl_model.describe_model()
retro_model = retro_rl_model.get_model(method, model_name)
retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game()
retro_rl_agent.save("latest", path='data')

In [ ]:
NN_path = retro_rl_agent.save("latest")

# Step 4: PostProcess Result

In [ ]:
# get the model you want to optimize
n_qubits = 8
device = 'local'
framework = 'pennylane'
shots = 100

model_name = "{}_{}_{}_{}".format(n_qubits, device, framework, shots)
method = "retro-qrl"

# train_mode can be: "local-instance", "local-job", "hybrid-job"
train_mode = "local-instance"

data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_model = None
if train_mode == "local-instance":
    # get model
    retro_rl_model = RetroRLModel.load(model_path)
    model_info = retro_rl_model.describe_model()
    retro_model = retro_rl_model.get_model(method, model_name)

retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()


In [ ]:
target = 'COC(Cc1ccc2oc(Cc3nc(-c4ccccc4)oc3C)cc2c1)OC'
retro_rl_agent.pathway(target)
layer2 = retro_rl_agent.layer2
# print(layer2)
path = set()
for i, j in layer2.items():
    for k, l  in j.items():
        path.add(l)
print(f"The path of retro_rl_agent: \n \
 {path}")

input_data_path = agent_param["data_path"]
ground_truth = np.load(input_data_path+'/ground_truth.npy', allow_pickle=True).tolist()

real_path = set(ground_truth[target]['path'])
print(f"The real_path of Brute force: \n \
 {real_path}")
real_cost = ground_truth[target]['cost']
print(f"The real_cost of Brute force: \n \
 {real_cost}")
print(f"Get the same path: {path == real_path}")

In [ ]:
target = 'O=C(NCc1ccc(CO)cc1)c1ccccn1'
retro_rl_agent.pathway(target)
layer2 = retro_rl_agent.layer2
# print(layer2)
path = set()
for i, j in layer2.items():
    for k, l  in j.items():
        path.add(l)
print(f"The path of retro_rl_agent: \n \
 {path}")

input_data_path = agent_param["data_path"]
ground_truth = np.load(input_data_path+'/ground_truth.npy', allow_pickle=True).tolist()

real_path = set(ground_truth[target]['path'])
print(f"The real_path of Brute force: \n \
 {real_path}")
real_cost = ground_truth[target]['cost']
print(f"The real_cost of Brute force: \n \
 {real_cost}")
print(f"Get the same path: {path == real_path}")

In [ ]:
target = 'CCCCC(CC)COC(=O)C(C#N)=C(c1ccccc1)c1ccccc1'
retro_rl_agent.pathway(target)
layer2 = retro_rl_agent.layer2
# print(layer2)
path = set()
for i, j in layer2.items():
    for k, l  in j.items():
        path.add(l)
print(f"The path of retro_rl_agent: \n \
 {path}")

input_data_path = agent_param["data_path"]
ground_truth = np.load(input_data_path+'/ground_truth.npy', allow_pickle=True).tolist()

real_path = set(ground_truth[target]['path'])
print(f"The real_path of Brute force: \n \
 {real_path}")
real_cost = ground_truth[target]['cost']
print(f"The real_cost of Brute force: \n \
 {real_cost}")
print(f"Get the same path: {path == real_path}")